# 0. Import lib

In [ ]:
import pandas as pd
import numpy as np

# 1. Read data from input file

In [ ]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# 2. Predefine function

In [ ]:
def find_weight(X, y):
    return np.round(np.array(np.linalg.inv(X.T @ X) @ X.T @ y), 3)
    
def MAE(y_predict, y_real):
    return np.mean(np.abs(y_predict - y_real))

In [ ]:
def next_state(state):
    for i in range(len(state)):
        if state[i] == 0:
            state[i] = 1
            break
        else:
            state[i] = 0
    return state

# 3. Choose suitable function

In [ ]:
with open("saved_result.txt", "r") as f:
    MAE_min = float(f.readline())
    state = np.fromstring(f.readline()[1:-1], dtype = np.int16, sep = " ")

res_state = None
stop_state = np.ones((23, ) , dtype = np.int16)

X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

while (1):
    next_state(state)
    index = np.where(state == 1)[0]
    
    w = find_weight(X_train_np[:, index], y_train_np)
    MAE_value = MAE(np.sum(X_test_np[:, index] * w.T, axis = 1) , y_test_np)
    
    if MAE_value < MAE_min:
        MAE_min = MAE_value
        res_state = state.copy()

        with open("saved_result.txt", "w") as f:
            f.write(str(MAE_min) + "\n")
            f.write(np.array2string(res_state, separator = " "))

        print("Found a better state: ", res_state)
    
    if np.array_equal(state, stop_state):
        break